## Evaluating my KG on OGB dataset

In [1]:
import pandas as pd

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch_geometric.utils import negative_sampling
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv

from kg_model import KG_model

from ogb.linkproppred import Evaluator, PygLinkPropPredDataset

from pykeen.evaluation import RankBasedEvaluator
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline 

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load and prepare data

In [2]:
dataset = PygLinkPropPredDataset(name='ogbl-ddi', transform=T.ToSparseTensor())
data = dataset[0]
data.adj_t

SparseTensor(row=tensor([   0,    0,    0,  ..., 4266, 4266, 4266]),
             col=tensor([   4,    6,    7,  ..., 3953, 3972, 4014]),
             size=(4267, 4267), nnz=2135822, density=11.73%)

In [3]:
split_edge = dataset.get_edge_split()
train_edge, valid_edge, test_edge = split_edge["train"], split_edge["valid"], split_edge["test"]
train_edge

{'edge': tensor([[4039, 2424],
         [4039,  225],
         [4039, 3901],
         ...,
         [ 647,  708],
         [ 708,  338],
         [ 835, 3554]])}

In [4]:
print(valid_edge['edge_neg'].shape)
print(valid_edge['edge'].shape)

torch.Size([101882, 2])
torch.Size([133489, 2])


In [5]:
def convert_to_triples_factory(data):
    tf_data = TriplesFactory.from_labeled_triples(
        data[["head", "relation", "tail"]].values,
        create_inverse_triples=True,
        entity_to_id=None,
        relation_to_id=None,
        compact_id=False 
    )

    print(tf_data.mapped_triples)

    return tf_data

In [6]:
# add relation type - interacts with

train = train_edge['edge']
train = torch.tensor([[x[0], 0, x[1]] for x in train])
train_df = pd.DataFrame(train, columns=['head', 'relation', 'tail']).astype(str)

valid = valid_edge['edge']
valid = torch.tensor([[x[0], 0, x[1]] for x in valid])
valid_df = pd.DataFrame(valid, columns=['head', 'relation', 'tail']).astype(str)

valid_neg = valid_edge['edge_neg']
valid_neg = torch.tensor([[x[0], 0, x[1]] for x in valid_neg])

test = test_edge['edge']
test = torch.tensor([[x[0], 0, x[1]] for x in test])
test_df = pd.DataFrame(test, columns=['head', 'relation', 'tail']).astype(str)

test_neg = test_edge['edge_neg']
test_neg = torch.tensor([[x[0], 0, x[1]] for x in test_neg])

train_tf = convert_to_triples_factory(train_df)
valid_tf = convert_to_triples_factory(valid_df)
test_tf = convert_to_triples_factory(test_df)

tensor([[   0,    0,  667],
        [   0,    0, 1182],
        [   0,    0, 1280],
        ...,
        [4266,    0, 4250],
        [4266,    0, 4252],
        [4266,    0, 4260]])
tensor([[   0,    0,  729],
        [   1,    0,  681],
        [   1,    0,  768],
        ...,
        [3812,    0, 3722],
        [3812,    0, 3758],
        [3812,    0, 3802]])
tensor([[   0,    0,    3],
        [   0,    0,  185],
        [   0,    0,  187],
        ...,
        [1611,    0, 1562],
        [1611,    0, 1573],
        [1611,    0, 1601]])


### Train my KG model

In [ ]:
model_kg = KG_model('TransE', train_tf, valid_tf, test_tf, 'ogb')
model_kg.set_params(20, 'Adam', RankBasedEvaluator, 'gpu')
print('Training...')
model_kg.train()
print('Training done')

### Compute scores for given triplets

In [8]:
# compute scores for positive and negative triplets 

batch_size = 512

n = train.size(0) // batch_size
pos_train_preds = []
for i in range(n+1):
    start_idx = i*batch_size
    end_idx = min((i+1)*batch_size, train.size(0))
    edge = train[start_idx:end_idx]
    pos_train_preds += [model_kg.trained_model.model.score_hrt(edge).squeeze().cpu().detach()]
pos_train_pred = torch.cat(pos_train_preds, dim=0)

n = valid.size(0) // batch_size
pos_valid_preds = []
for i in range(n+1):
    start_idx = i*batch_size
    end_idx = min((i+1)*batch_size, valid.size(0))
    edge = valid[start_idx:end_idx]
    pos_valid_preds += [model_kg.trained_model.model.score_hrt(edge).squeeze().cpu().detach()]
pos_valid_pred = torch.cat(pos_valid_preds, dim=0)

n = valid_neg.size(0) // batch_size
neg_valid_preds = []
for i in range(n+1):
    start_idx = i*batch_size
    end_idx = min((i+1)*batch_size, valid_neg.size(0))
    edge = valid_neg[start_idx:end_idx]
    neg_valid_preds += [model_kg.trained_model.model.score_hrt(edge).squeeze().cpu().detach()]
neg_valid_pred = torch.cat(neg_valid_preds, dim=0)

n = test.size(0) // batch_size
pos_test_preds = []
for i in range(n+1):
    start_idx = i*batch_size
    end_idx = min((i+1)*batch_size, test.size(0))
    edge = test[start_idx:end_idx]
    pos_test_preds += [model_kg.trained_model.model.score_hrt(edge).squeeze().cpu().detach()]
pos_test_pred = torch.cat(pos_test_preds, dim=0)

n = test_neg.size(0) // batch_size
neg_test_preds = []
for i in range(n+1):
    start_idx = i*batch_size
    end_idx = min((i+1)*batch_size, test_neg.size(0))
    edge = test_neg[start_idx:end_idx]
    neg_test_preds += [model_kg.trained_model.model.score_hrt(edge).squeeze().cpu().detach()]
neg_test_pred = torch.cat(neg_test_preds, dim=0)

### Evaluate my results

In [9]:
# Evaluate the coputed scores - hits@K

evaluator = Evaluator(name = 'ogbl-ddi')

results = {}
for K in [10, 20, 30]:
    evaluator.K = K
    train_hits = evaluator.eval({
        'y_pred_pos': pos_train_pred,
        'y_pred_neg': neg_valid_pred,
    })[f'hits@{K}']
    valid_hits = evaluator.eval({
        'y_pred_pos': pos_valid_pred,
        'y_pred_neg': neg_valid_pred,
    })[f'hits@{K}']
    test_hits = evaluator.eval({
        'y_pred_pos': pos_test_pred,
        'y_pred_neg': neg_test_pred,
    })[f'hits@{K}']
    
    results[f'Hits@{K}'] = (train_hits, valid_hits, test_hits)
    
    
for hits, result in results.items():
    print(hits)
#     print(result)
    train_hits, valid_hits, test_hits = result
    print(f'Train: {100 * train_hits:.2f}%')
    print(f'Valid: {100 * valid_hits:.2f}%')
    print(f'Test: {100 * test_hits:.2f}%')


Hits@10
Train: 0.01%
Valid: 0.01%
Test: 0.01%
Hits@20
Train: 0.02%
Valid: 0.02%
Test: 0.02%
Hits@30
Train: 0.03%
Valid: 0.03%
Test: 0.03%


In [31]:
model_kg.predict_head('2424', '0')

Leuprolide - decrease_adverse_effects:
      head_id head_label     score  in_training
1776     1776       2597 -4.225868         True
1787     1787       2606 -4.259010         True
1549     1549       2392 -4.277555         True
3626     3626       4261 -4.303725         True
4148     4148        892 -4.309386         True
1917     1917       2723 -4.312338        False
1929     1929       2734 -4.312355         True
1610     1610       2447 -4.334918         True
3955     3955        718 -4.335896        False
1236     1236        211 -4.366917         True


In [32]:
print(model_kg.trained_model.get_metric('hits@1'))
print(model_kg.trained_model.get_metric('hits@5'))
print(model_kg.trained_model.get_metric('hits@10'))

0.0005993003168800426
0.0036557319329682597
0.007348920135741521


### Optimizing parameters

In [9]:
from pykeen.hpo import hpo_pipeline_from_config

config = {
    'optuna': dict(
        n_trials=5,
    ),
    'pipeline': dict(
#         dataset='Nations',
        training = train_tf,
        testing = test_tf,
        validation = valid_tf,
        model='TransE',
        model_kwargs_ranges=dict(
               embedding_dim=dict(type=int, low=50, high=220, q=20),
        ),
        optimizer='Adam',
        optimizer_kwargs=dict(lr=0.01),
        loss='marginranking',
        loss_kwargs=dict(margin=1),
        training_loop='slcwa',
        training_kwargs=dict(num_epochs=20, batch_size=128),
        negative_sampler='basic',
        negative_sampler_kwargs=dict(num_negs_per_pos=1),
        evaluator_kwargs=dict(filtered=True),
        evaluation_kwargs=dict(batch_size=128),
        stopper='early',
        stopper_kwargs=dict(frequency=5, patience=2, relative_delta=0.002),
    )
}

In [10]:
hpo_pipeline_result = hpo_pipeline_from_config(config)

[I 2023-01-30 08:57:20,367] A new study created in memory with name: no-name-c73c846a-6d4b-4b64-a2f9-afaca0b6b610
/usr/local/lib/python3.8/dist-packages/optuna/distributions.py:683: UserWarning: The distribution is specified by [50, 220] and step=20, but the range is not divisible by `step`. It will be replaced by [50, 210].
  warnings.warn(
No random seed is specified. Setting to 1486788349.
INFO:pykeen.triples.triples_factory:Creating inverse triples.
Training epochs on cuda:0:   0%|                                               | 0/20 [00:00<?, ?epoch/s]INFO:pykeen.triples.triples_factory:Creating inverse triples.

Training batches on cuda:0:  24%|███████▎                       | 3948/16687 [00:06<00:21, 583.89batch/s]


Training batches on cuda:0:  50%|███████████████▌               | 8346/16687 [00:14<00:14, 589.99batch/s]


Training batches on cuda:0:  77%|███████████████████████       | 12843/16687 [00:22<00:06, 557.75batch/s]


Training batches on cuda:0:   3%|▊                               | 427/16687 [00:00<00:28, 576.58batch/s]


Training batches on cuda:0:  29%|█████████                      | 4905/16687 [00:08<00:19, 591.51batch/s]


...

Training batches on cuda:0:  11%|███▎                           | 1765/16687 [00:03<00:25, 580.03batch/s]


Training batches on cuda:0:  37%|███████████▍                   | 6173/16687 [00:10<00:17, 588.37batch/s]


Training batches on cuda:0:  63%|██████████████████▉           | 10558/16687 [00:18<00:10, 583.47batch/s]


Training epochs on cuda:0:  40%|████      | 8/20 [05:03<07:35, 37.94s/epoch, loss=0.701, prev_loss=0.701]
[W 2023-01-30 09:02:25,021] Trial 0 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/usr/local/lib/python3.8/dist-packages/pykeen/hpo/hpo.py", line 259, in __call__
    result = pipeline(
  File "/usr/local/lib/python3.8/dist-packages/pykeen/pipeline/api.py", line 1291, in pipeline
    losses = training_loop_instance.train(
  File "/usr/local/lib/python3.8/dist-packages/pykeen/training/training_loop.py", line 378, in train
    result = self._train(
  File "/usr/local/lib/python3.8/dist-packages/pykeen/training/training_loop.py", line 659, in _train
    self.optimizer.step()
  File "/usr/local/lib/python3.8/dist-packages/torch/optim/optimizer.py", line 140, in wrapper
    out = func(*args, **kwargs)

KeyboardInterrupt: 

In [ ]:
hpo_pipeline_result.save_to_directory('hpo_results')

### Example from OGB

In [22]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels, cached=True))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, cached=True))
        self.convs.append(GCNConv(hidden_channels, out_channels, cached=True))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x

In [23]:
class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(LinkPredictor, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()

    def forward(self, x_i, x_j):
        x = x_i * x_j
        for lin in self.lins[:-1]:
            x = lin(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.sigmoid(x)

In [24]:
def train(model, predictor, x, adj_t, split_edge, optimizer, batch_size):

    row, col, _ = adj_t.coo()
    edge_index = torch.stack([col, row], dim=0)

    model.train()
    predictor.train()

    pos_train_edge = split_edge['train']['edge'].to(x.device)

    total_loss = total_examples = 0
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size,
                           shuffle=True):
        optimizer.zero_grad()

        h = model(x, adj_t)
#         print('h:', h)

        edge = pos_train_edge[perm].t()
#         print('Train: edge:', edge)
#         print()
#         print('h[edge[0]]:', h[edge[0]])
#         print()
#         print('h[edge[1]]:', h[edge[1]])
#         print()

        pos_out = predictor(h[edge[0]], h[edge[1]])
        pos_loss = -torch.log(pos_out + 1e-15).mean()
#         print('pos out:', pos_out)

        edge = negative_sampling(edge_index, num_nodes=x.size(0),
                                 num_neg_samples=perm.size(0), method='dense')

        neg_out = predictor(h[edge[0]], h[edge[1]])
        neg_loss = -torch.log(1 - neg_out + 1e-15).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(x, 1.0)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples

In [29]:
@torch.no_grad()
def test(model, predictor, x, adj_t, split_edge, evaluator, batch_size):
#     print('test')
    
    model.eval()
    predictor.eval()

    h = model(x, adj_t)

    pos_train_edge = split_edge['eval_train']['edge'].to(x.device)
    pos_valid_edge = split_edge['valid']['edge'].to(x.device)
    neg_valid_edge = split_edge['valid']['edge_neg'].to(x.device)
    pos_test_edge = split_edge['test']['edge'].to(x.device)
    neg_test_edge = split_edge['test']['edge_neg'].to(x.device)

    pos_train_preds = []
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size):
        edge = pos_train_edge[perm].t()
        pos_train_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_train_pred = torch.cat(pos_train_preds, dim=0)

    pos_valid_preds = []
    for perm in DataLoader(range(pos_valid_edge.size(0)), batch_size):
        edge = pos_valid_edge[perm].t()
        pos_valid_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_valid_pred = torch.cat(pos_valid_preds, dim=0)

    neg_valid_preds = []
    for perm in DataLoader(range(neg_valid_edge.size(0)), batch_size):
        edge = neg_valid_edge[perm].t()
        neg_valid_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    neg_valid_pred = torch.cat(neg_valid_preds, dim=0)

    pos_test_preds = []
    for perm in DataLoader(range(pos_test_edge.size(0)), batch_size):
        edge = pos_test_edge[perm].t()
        pos_test_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_test_pred = torch.cat(pos_test_preds, dim=0)

    neg_test_preds = []
    for perm in DataLoader(range(neg_test_edge.size(0)), batch_size):
        edge = neg_test_edge[perm].t()
        neg_test_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    neg_test_pred = torch.cat(neg_test_preds, dim=0)
    
#     print('pos_train_pred:', pos_train_pred)
#     print('neg_train_pred:', neg_valid_pred)
#     print()

    results = {}
    for K in [10, 20, 30]:
        evaluator.K = K
        train_hits = evaluator.eval({
            'y_pred_pos': pos_train_pred,
            'y_pred_neg': neg_valid_pred,
        })[f'hits@{K}']
        valid_hits = evaluator.eval({
            'y_pred_pos': pos_valid_pred,
            'y_pred_neg': neg_valid_pred,
        })[f'hits@{K}']
        test_hits = evaluator.eval({
            'y_pred_pos': pos_test_pred,
            'y_pred_neg': neg_test_pred,
        })[f'hits@{K}']

        results[f'Hits@{K}'] = (train_hits, valid_hits, test_hits)

    return results

In [30]:
hidden_channels = 256
num_layers = 2
dropout = 0.5
runs = 4
lr = 0.005
batch_size = 64 * 1024
epochs = 5
log_steps = 1
eval_steps = 1

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

dataset = PygLinkPropPredDataset(name='ogbl-ddi', transform=T.ToSparseTensor())
data = dataset[0]
adj_t = data.adj_t.to(device)

split_edge = dataset.get_edge_split()

# We randomly pick some training samples that we want to evaluate on:
torch.manual_seed(12345)
idx = torch.randperm(split_edge['train']['edge'].size(0))
idx = idx[:split_edge['valid']['edge'].size(0)]
split_edge['eval_train'] = {'edge': split_edge['train']['edge'][idx]}


model = GCN(hidden_channels, hidden_channels,
                hidden_channels, num_layers,
                dropout).to(device)

emb = torch.nn.Embedding(data.adj_t.size(0),
                         hidden_channels).to(device)

print('Embedding:', emb)
print()
predictor = LinkPredictor(hidden_channels, hidden_channels, 1,
                          num_layers, dropout).to(device)

evaluator = Evaluator(name='ogbl-ddi')
# loggers = {
#     'Hits@10': Logger(args.runs, args),
#     'Hits@20': Logger(args.runs, args),
#     'Hits@30': Logger(args.runs, args),
# }

for run in range(runs):
    torch.nn.init.xavier_uniform_(emb.weight)
#     print('Weights:', emb.weight)
#     print()
    model.reset_parameters()
    predictor.reset_parameters()
    optimizer = torch.optim.Adam(
        list(model.parameters()) + list(emb.parameters()) +
        list(predictor.parameters()), lr=lr)

    for epoch in range(1, 1 + epochs):
        loss = train(model, predictor, emb.weight, adj_t, split_edge,
                     optimizer, batch_size)

        if epoch % eval_steps == 0:
#             print('Eval')
            results = test(model, predictor, emb.weight, adj_t, split_edge,
                           evaluator, batch_size)
#             for key, result in results.items():
#                 loggers[key].add_result(run, result)

            if epoch % log_steps == 0:
                for key, result in results.items():
                    train_hits, valid_hits, test_hits = result
                    print(key)
                    print(f'Run: {run + 1:02d}, '
                          f'Epoch: {epoch:02d}, '
                          f'Loss: {loss:.4f}, '
                          f'Train: {100 * train_hits:.2f}%, '
                          f'Valid: {100 * valid_hits:.2f}%, '
                          f'Test: {100 * test_hits:.2f}%')
                print('---')

#     for key in loggers.keys():
#         print(key)
#         loggers[key].print_statistics(run)

# for key in loggers.keys():
#     print(key)
#     loggers[key].print_statistics()

Embedding: Embedding(4267, 256)

Hits@10
Run: 01, Epoch: 01, Loss: 1.2921, Train: 0.03%, Valid: 0.02%, Test: 0.01%
Hits@20
Run: 01, Epoch: 01, Loss: 1.2921, Train: 3.96%, Valid: 3.64%, Test: 2.58%
Hits@30
Run: 01, Epoch: 01, Loss: 1.2921, Train: 4.55%, Valid: 4.17%, Test: 3.92%
---
Hits@10
Run: 01, Epoch: 02, Loss: 0.9923, Train: 3.10%, Valid: 2.87%, Test: 5.68%
Hits@20
Run: 01, Epoch: 02, Loss: 0.9923, Train: 4.74%, Valid: 4.37%, Test: 6.92%
Hits@30
Run: 01, Epoch: 02, Loss: 0.9923, Train: 5.45%, Valid: 5.01%, Test: 7.74%
---
Hits@10
Run: 01, Epoch: 03, Loss: 0.8506, Train: 0.26%, Valid: 0.24%, Test: 0.03%
Hits@20
Run: 01, Epoch: 03, Loss: 0.8506, Train: 0.56%, Valid: 0.51%, Test: 0.08%
Hits@30
Run: 01, Epoch: 03, Loss: 0.8506, Train: 0.75%, Valid: 0.70%, Test: 0.15%
---
Hits@10
Run: 01, Epoch: 04, Loss: 0.7528, Train: 2.34%, Valid: 2.15%, Test: 3.71%
Hits@20
Run: 01, Epoch: 04, Loss: 0.7528, Train: 3.19%, Valid: 2.99%, Test: 4.94%
Hits@30
Run: 01, Epoch: 04, Loss: 0.7528, Train: 4.13